모나드를 사용하면 모나드를 사용하지 않은 겨우 식을 다양한 순서로 평가할 수 있는 언어에서 평가 순서를 지정할 수 있다. 모나드를 사용해 a+b+c와 같은 식이 왼쪽에서 오른쪽 순서로 평가되게 만들 수 있다. 일반적으로는 이러한 모나드의 기능이 그리 유용해 보이지 않는다. 하지만 파일의 내용을 특정 순서대로 쓰고 읽고 싶은 경우, 모나드를 사용하면 read와 write 함수가 특정 순서대로 평가되도록 쉽게 만들 수 있다.

평가 순서가 다양하고, 최적화 컴파일러를 제공하는 언어의 경우 모나드를 사용하면 식을 평가하는 순서를 정할 수 있다는 장점이 있다. 파이썬의 경우 대부분의 부분이 엄격한 평가를 수행하고 최적화를 수행하지 않는다. 따라서 모나드를 실용적으로 활용할 수 있는 가능성을 별로 없다. 

그러나 PyMonad 모듈은 모나드 이상이다. 그 안에는 별도의 구현이 있는 함수형 프로그래밍 기능이 많이 들어 있다. PyMonad 모듈을 사용하면 표준 라이브러리에 있는 모듈만을 사용할 때 보다 더 간결하고 이해하기 쉬운 프로그램을 만들 수 있는 경우가 있다.

In [2]:
import pymonad

In [3]:
help(pymonad)

Help on package pymonad:

NAME
    pymonad

PACKAGE CONTENTS
    either
    io
    list
    maybe
    monad
    monoid
    operators (package)
    promise
    reader
    state
    tools
    writer

FILE
    c:\programdata\anaconda3\lib\site-packages\pymonad\__init__.py




### 함수적 합성과 커링

일부 함수형 언어는 인자가 많은 함수를 인자가 하나뿐인 여러 함수로 변환하는 방식으로 작동한다. 이러한 처리 과정을 커링이라 한다. 

파이썬에서 구체적인 예를 살펴보자. 다음과 같은 함수가 있다고 가정하자.

In [24]:
from pymonad.tools import curry

@curry(4)
def systolic_bp(bmi, age, gender_male, treatment):
    return 68.15+0.58*bmi+0.65*age+0.94*gender_male+6.44*treatment

이 함수는 수축기 혈압을 기반으로 한 간단한 다중 회귀 기반 모델이다.

다음과 같이 systolic_bp 함수를 네 가지 이자를 모두 넘겨 사용할 수 있다.

In [27]:
systolic_bp(25, 50, 1, 0)

116.09

In [28]:
systolic_bp(25, 50, 0, 1)

121.59

In [29]:
treated = systolic_bp(25, 50, 0)
treated(0)

115.15

In [30]:
treated(1)

121.59

In [31]:
g_t = systolic_bp(25, 50)

In [33]:
g_t(1, 0)

116.09

In [34]:
g_t(0, 1)

121.59

### 커링한 고차 함수 사용하기

평범한 함수를 사용하면 커링을 보여주기 쉽지만, 실제 커링의 카치는 고차 함수에 적용하는 경우 나타난다. functools.reduce 함수는 커링가능해야 할 것이며, 다음과 같이 사용할 수도 있을 것이다.

In [42]:
from functools import reduce
import operator
a_list = [1, 2, 3, 4, 5]
sums = reduce(operator.add, a_list)

In [44]:
sums

15

하지만 reduce 함수는 커링이 가능하지 않다. 따로 reduce 함수를 정의한다면 커링을 허용할 수 있다.

In [57]:
import collections.abc

@curry(2)
def user_defined_reduce(function, iterable_or_sequece):
    if isinstance(iterable_or_sequece, collections.abc.Sequence):
        iterator = iter(iterable_or_sequece)
    else:
        iterator = iterable_or_sequece
    s = next(iterator)
    for v in iterator:
        s = function(s, v)
    return s

In [58]:
ud_sum = user_defined_reduce(operator.add)

In [59]:
ud_sum([1, 2, 3])

6

In [60]:
ud_max = user_defined_reduce(lambda x, y: x if x> y else y)

In [61]:
ud_max([2, 1, 5])

5

하지만 위와 같은 방식은 일반적인 max 함수를 쉽게 구현할 수 없다. 커링은 위치 기반 매개변수에 의존하기 때문이다. key라는 키워드 기반 매개변수를 지정할 수 있게 만들려면 구현이 너무 복잡해져서 간결하면서 이해하기 쉬운 함수형 프로그램이라는 목적에 위배된다.

더 일반적인 max 함수를 만들려면 key라는 키워드 기반 매개변수라는 패러다임에서 벗어나야 한다. map filter reduce 함수 등이 하는 것과 같이 정렬 키를 뽑아내는 함수를 첫 번째 인자로 받는 것을 인정해야 할 것이다. 

### 더 어려운 방식으로 커링하기

커링한 함수를 라이브러리 데코레이터 없이 직접 작성할 수도 있다. 

In [62]:
def f(x, *args):
    def f1(y, *args):
        def f2(z):
            return (x+y)*z
        if args:
            return f2(*args)
        return f2
    if args:
        return f1(*args)
    return f1

이러한 방식은 실수하기 쉽다. 하지만 이 예제는 커링이 진정 의미하는 것이 무엇이고, 어떻게 파이썬으로 커링을 구현할 수 있는지 잘 보여준다. 

### 함수적 합성과 PyMonad 곱셈 연산자

커링한 함수의 중요한 가치 중 하나는 함수적 합성을 통해 그러한 함수를 서로 조합할 수 있다는 점이다. 

커링한 함수를 만들면, 함수적 합성을 사용하여 쉽게 더 복잡한 커링한 함수를 새로 만들 수 있다. pymonad 라이브러리에서는 * 연산자를 사용해 두 함수를 합성한다.

In [63]:
import operator
prod = user_defined_reduce(operator.mul)

다음은 값의 범위를 계산하는 두 번째 커리한 함수다.

In [64]:
@curry(1)
def alt_range(n):
    if n == 0: return range(1, 2)
    if n % 2 == 0:
        return range(2, n+1, 2)
    else:
        return range(1, n+2, 2)

다음은 prod와 alt_range를 합성하여 새로운 커링한 함수를 만드는 방법을 보여준다. 라이브러리 버전에 따라 Compose로 대체해야 한다.

In [73]:
from pymonad.reader import Compose

# semi_fact = prod * alt_range

semi_fact = (Compose(alt_range)
            .then(prod))

In [74]:
semi_fact(9)

945

이를 파이썬에서 직접 구현한 경우 람다로 정의할 수 있다.

In [75]:
semi_fact = lambda x: prod(alt_range(x))

이상적인 경우, 함수적 합성과 커링한 함수를 다음과 같이 사용할 수 있으면 좋을 것이다.

In [78]:
sumwhile = sum * takewhile(lambda x: x > 1E-7)

이렇게 하면 무한 시퀀스에 대해 작동하면서 생성되는 값이 주어진 기준을 만족하는 동안만 값을 만들어 낼 수 있는 sum 함수를 정의할 수 있다. 하지만 Pymonad 라이브러리가 List 객체를 처리하는 것만큼 무한 반복 가능 객체를 잘 다루지 못하기 때문에 이러한 코드가 잘 작동하지 않는 것 같다.

### 펑터와 적용 가능 펑터(functor)

펑터는 간단한 데이터를 함수로 표현한다는 개념이다. 예제를 보자.

In [79]:
pi = lambda : 3.14

단순한 값을 돌려주는 인자가 없는 람다 객체이다. 

키러항 함수를 펑터에 적용하면 새로운 커링한 펑터가 생긴다. 이 방법은 함수를 인자에 적용해 결과 값을 얻는다는 개념을 확장하여 함수가 인자 값 역할도 하고 함수 역할도 하게 만든 것이다. 

프로그램에 있는 모든 것이 함수이기 때문에 모든 처리는 함수 합성이라는 주제에 대한 변주일 뿐이다. 커링한 함수의 인자와 결콰는 펑터일 수 있다. 

모든 것이 함수 합성이기 때문에 실제로 메서드를 사용해 값을 요청하기 전까지는 아무런 계산도 수행할 필요가 없다. 펑터를 사용한 프로그램은 복잡한 계산을 수행하지 않고, 요청을 받으면 값을 만들어 내는 복잡한 객첼을 정의한다. 이론적으로는 똑똑한 컴파일러나 런파임 시스템이 있다면 이러한 합성을 최적화시켜줄 수 있다. 

함수를 펑터 객체에 적용하는 것은 * 연산자롤 구현된 map과 비슷한 메서드를 사용하려는 것이다. 펑터가 식에서 어떤 역할을 하는지 이해하기 위해 함수 * 펑터나 map(함수, 펑터)를 생각할 수 있다. 

인자가 여럿 있는 ㅎ마수와 잘 어우러지기 위해 합성 펑터를 만드는 & 연산자를 사용한다. 한 쌍의 펑터로부터 새로운 펑터 객체를 만들기 위해 펑터 & 펑터를 자주 사용할 것이다. 

파이썬의 단순한 타입을 Maybe 펑터로 감쌀 수 있다. Maybe 펑터는 데이터가 없는 경우를 우아하게 처리할 수 있기 때문에 흥미롭다. pymonad에서 사용하는 접근 방식은 데이터를 감싸서 문제가 생겨도 잘 처리될 수 있게 만드는 방식이다. 

Maybe 펑터에는 두 가지 하위 클래스가 있다.

* Nothing
* Just(단순한 값)

여기서는 None 대신 Nothing을 사용한다. 이것이 사용할 수 없는 데이터를 표현한다. Just를 사용해 None이 아닌 모든 파이썬 객체를 감싼다. 이러한 펑터는 상수 값을 함수와 유사하게 표현한 것이다.

커링한 함수를 Maybe 객체에 사용하면 데이터를 사용할 수 없는 경우에도 잘 처리할 수 있다.

### 지연 List 펑터 사용하기

List 펑터는 파이썬의 내장 List타입과 달리, 극단적으로 지연 계산을 활용한다. 내장 list(range(10))을 사용하면 list() 함수가 range() 객체를 평가하여 원소가 10개인 리스트를 만든다. 하지만 pymonad의 List 펑터는 지연 계산을 수행하기 때문에 이러한 평가를 전혀 수행하지 않는다.

In [89]:
from pymonad.tools import List

list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [98]:
help(List)

Help on _GenericAlias in module typing object:

class _GenericAlias(_Final)
 |  _GenericAlias(origin, params, *, inst=True, special=False, name=None)
 |  
 |  The central part of internal API.
 |  
 |  This represents a generic version of type 'origin' with type arguments 'params'.
 |  There are two kind of these aliases: user defined and special. The special ones
 |  are wrappers around builtin collections and ABCs in collections.abc. These must
 |  have 'name' always set. If 'inst' is False, then the alias can't be instantiated,
 |  this is used by e.g. typing.List and typing.Dict.
 |  
 |  Method resolution order:
 |      _GenericAlias
 |      _Final
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getattr__(self, attr)
 |  
 |  __getitem__(self, params)
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, origin, p

List 펑터는 range 객체를 평가하지 않고 나중에 사용하기 위해 저장해두기만 한다. 함수는 함수를 평가하지 않고 컬렉션을 모으고 싶을 떄 유용하다. 

다음은 range 함수를 커링한 버전이다. 0이 아니라 1이 최솟값으로 했다. 

In [99]:
@curry(1)
def range1n(n):
    if n == 0: return range(1, 2)
    return range(1, n+1)

In [100]:
range1n(10)

range(1, 11)

다음은 예제를 확장하기 위해 사용할 수 있는 다른 간단한 함수다.

In [101]:
@curry(1)
def n21(n):
    return 2*n+1

In [103]:
f2 = (Compose(semi_fact).then(n21))

### 모나드의 개념과 bind 함수, 이진 오른쪽 시프트 연산자

모나드는 엄격한 순서가 있다. 함수형 프로그래밍은 순서를 자유롭게 정할 수 있다. 그래서 필요에 따라 함수 호출 순서를 바꿀 수 있다. 
모나드를 사용하면 엄격하게 왼쪽에서 오른쪽으로 계산을 수행할 수 있다는 점에서 함수형 프로그래밍에 대한 예외라고 할 수 있다. 

bind 함수를 >> 연산자에 연결해뒀기 때문에 다음과 같은 식을 작성할 수 있다.

Just(some file) >> read header >> read next >> read next

앞의 식을 변환하면 다음과 같다.

bind(bind(bind(Just(some file), read header), read next), read next)

### 모나드를 사용해 시뮬레이션 구현하기

모나드는 일종의 파이프라인을 통해 전달되는 대상이다. 어떤 모나드를 함수에 넘기면 비슷한 모나드가 결과 값으로 나온다. 

어떤 처리 과정을 시뮬레이션 할 때 사용할 수 있는 간단한 파이프라인을 살펴본다. 이는 몬테카를로 시뮬레이션의 일부분일 수 있다. 

난수를 발생하는 함수

In [104]:
import random

def rng():
    return (random.randint(1, 6, random.randint(1, 6)))

전체 게임이 어떻게 흘러갈지 보여준다.

In [114]:
def craps():
    outcome = Just(("", 0, []) ) >> come_out_roll(rng) >> point_roll(rng)
    print(outcome.getValue())

come_out_roll 함수

In [107]:
@curry(2)
def come_out_roll(dice, status):
    d = dice()
    if sum(d) in (7, 11):
        return Just(("win", sum(d), [d]))
    elif sum(d) in (2, 3, 12):
        return Just(("lose", sum(d), [d]))
    else:
        return Just(("point", sum(d) [d]))

point_roll 함수

In [112]:
@curry(2)
def point_roll(dice, status):
    prev, point, so_far = status
    if prev != "point":
        return Just(status)
    d = dice()
    if sum(d) == 7:
        return Just(("craps", point, so_far+[d]))
    elif sum(d) == point:
        return Just(("win", point, so_far+[d]))
    else:
        return Just(("point", point, so_far+[d])) >> point_roll(dice)

In [116]:
craps()

영리한 몬테 카를로 시뮬레이션의 상당 부분을 몇 가지 단순한 함수형 프로그래밍 설계 기법으로 만들 수 있다. 특히, 모나드를 사용하면 내부 상태가 있거나 복잡한 선후 관계가 있는 경우에 계산을 구조화하는 데 도움이 될 수 있다.